https://www.youtube.com/watch?v=QVGVL3AGmxk

#tuto taken from this video

In [22]:
from common.keys import MISTRAL_API_KEY


MISTRAL_API_KEY

'0lWmBpHUHkxqTu5TnEbdHkJzCj7vgq4u'

In [23]:

from cocoAI.folder_tree import get_enseigne_folder
from common.identifiers import convert_to_siren
siret = "53446191800037"
siren = convert_to_siren(siret)
ENSEIGNE_FOLDER = get_enseigne_folder(siret)
MISTRAL_PATH = list(ENSEIGNE_FOLDER.rglob("**/MISTRAL_FILES/"))[0]
FILES_TO_ANALYSE = list(ENSEIGNE_FOLDER.rglob("BULL*/*.pdf"))

In [24]:
from mistralai import Mistral
client = Mistral(api_key=MISTRAL_API_KEY)


In [25]:
import base64
import json
from common.AI_API import ask_Mistral
from common.keys import MISTRAL_API_KEY_PAYANTE
from common.logconfig import LOGGER


def data_uri_to_bytes(data_uri):
    _, encoded = data_uri.split(",", 1)
    return base64.b64decode(encoded).decode("utf-8")


def export_image(image):
    parsed_image = data_uri_to_bytes(image.image_base64)
    with open(image.id, "wb", encoding="utf-8") as file:
        file.write(parsed_image)


def parse_pdf(file_path):
    uploaded_file = client.files.upload(
        file={"file_name": str(file_path), "content": open(str(file_path), "rb")},
        purpose="ocr",
    )
    file_url = client.files.get_signed_url(file_id=uploaded_file.id)
    response = client.ocr.process(
        model="mistral-ocr-latest",
        document={"type": "document_url", "document_url": file_url.url},
        include_image_base64=True,
    )

    md_output_path = MISTRAL_PATH/(file_path.stem+"_output.md")
    with open(md_output_path, "w", encoding="utf-8") as f:
        for page in response.pages:
            f.write(page.markdown)
            for image in page.images:
                export_image(image)
    print(md_output_path)
    return md_output_path


def output_employes_json(request, output_path):

    # response_dict = json.loads(ocr_response.model_dump_json())
    # ocr_markdown = response_dict["pages"][0]["markdown"]

    request_file = output_path.with_suffix(".request")
    with open(request_file, "w", encoding="utf-8") as f:
        f.write(request)
        LOGGER.debug(f"request exported to {request_file}")

    model = "mistral-large-latest"
    chat_response = ask_Mistral(
        api_key=MISTRAL_API_KEY_PAYANTE,
        prompt=request,
        model=model,
        json_only=True,
    )
    json_dict = chat_response.choices[0].message.content

    response_dict2 = json.loads(json_dict)
    json_path = output_path.with_suffix(".json")
    with open(json_path, "w", encoding="utf-8") as f:
        f.write(json.dumps(response_dict2, ensure_ascii=False))
        LOGGER.debug(json_path.resolve())
    print(json_path)
    
    return response_dict2

In [26]:
# from pprint import pprint
# pprint(response.pages[0])
# response.pages[0].markdown

In [27]:
for sal in FILES_TO_ANALYSE:
    print(sal)
    md_output = parse_pdf(sal)
    
    with open(md_output,'r',encoding='utf-8') as f:
        ocr_output = '\n'.join(f.readlines())
    
    request = """pour le document joint, extrais : 
- les salaires net à payer avant impôt sur le revenu
- les dates
- les mois concernés
- les noms des employés
- les postes occupés
- les anciennetés
 Présente le résultat sous forme d'un dictionnaire json de la forme 
 "Employés": [
        {
            "Nom": ,
            "Poste": 
            "Salaire net à payer avant impôt sur le revenu": 
            "Date d'entrée":
            "Ancienneté":
            "Mois concernés": pd.datetime().strftime("%Y-%m")
        },
 
 
 \n"""+ocr_output 
 
    di = output_employes_json(
        request, MISTRAL_PATH / (sal.name + "salaires")
    )


C:\Users\lvolat\COMPTOIRS ET COMMERCES\DATALAKE - Documents\COVI_534461918\LE_RICHER\REFERENCE_DOCUMENTS\SOCIAL\BULLETINS_PAIE\0824_COVI_CDI.pdf
C:\Users\lvolat\COMPTOIRS ET COMMERCES\DATALAKE - Documents\COVI_534461918\LE_RICHER\MISTRAL_FILES\0824_COVI_CDI_output.md
C:\Users\lvolat\COMPTOIRS ET COMMERCES\DATALAKE - Documents\COVI_534461918\LE_RICHER\MISTRAL_FILES\0824_COVI_CDI.json


In [28]:
di

{'Employés': [{'Nom': 'AMANE SARAH',
   'Poste': 'DIRECTRICE DE SALLE',
   'Salaire net à payer avant impôt sur le revenu': 2719.66,
   "Date d'entrée": '24/11/2023',
   'Ancienneté': '9 mois',
   'Mois concernés': '2024-08'},
  {'Nom': 'DAVID QUENTIN',
   'Poste': 'CHEF DE CUISINE',
   'Salaire net à payer avant impôt sur le revenu': 3235.77,
   "Date d'entrée": '27/09/2021',
   'Ancienneté': '2 ans et 11 mois',
   'Mois concernés': '2024-08'},
  {'Nom': "DINGUI D'EMILS",
   'Poste': 'SERVEUR',
   'Salaire net à payer avant impôt sur le revenu': 2090.08,
   "Date d'entrée": '01/03/2024',
   'Ancienneté': '6 mois',
   'Mois concernés': '2024-08'},
  {'Nom': 'JABET Martin',
   'Poste': 'APPRENTI CUISINE',
   'Salaire net à payer avant impôt sur le revenu': 1013.51,
   "Date d'entrée": '01/09/2023',
   'Ancienneté': '1 an',
   'Mois concernés': '2024-08'},
  {'Nom': 'KATHIRESU SRIKANTA',
   'Poste': 'PLONGEUR',
   'Salaire net à payer avant impôt sur le revenu': 1881.89,
   "Date d'entré

In [29]:
import pandas as pd


df = pd.DataFrame(di['Employés'])

In [33]:
df = df.rename(columns={'Salaire net à payer avant impôt sur le revenu':'Salaire brut'})

In [32]:
df

,Nom,Poste,Salaire Brut,Date d'entrée,Ancienneté,Mois concernés
0,AMANE SARAH,DIRECTRICE DE SALLE,2719.66,24/11/2023,9 mois,2024-08
1,DAVID QUENTIN,CHEF DE CUISINE,3235.77,27/09/2021,2 ans et 11 mois,2024-08
2,DINGUI D'EMILS,SERVEUR,2090.08,01/03/2024,6 mois,2024-08
3,JABET Martin,APPRENTI CUISINE,1013.51,01/09/2023,1 an,2024-08
4,KATHIRESU SRIKANTA,PLONGEUR,1881.89,01/07/2022,2 ans et 2 mois,2024-08
5,LANCINA LOHAN,APPRENTI CUISINE,626.27,31/08/2022,2 ans,2024-08
6,MEIRIM ANAELLE,SERVEUSE,1423.70,03/04/2024,5 mois,2024-08
7,MOHAMMAD AZIR UDDIN,COMMIS DE CUISINE,2082.23,02/07/2022,2 ans et 1 mois,2024-08
8,MOHAMMAD HASANUZZAMAN,PLONGEUR,1849.30,01/07/2023,1 an et 4 mois,2024-08
9,RICHARD ELISE,SERVEUSE,1421.30,01/05/2024,6 mois,2024-08


In [34]:
df.to_csv(MISTRAL_PATH / "tableau_salaires.csv")